<font size="6">Let's predict the new world champion..!</font>

'

In [21]:
import pandas as pd
import pickle 
from scipy.stats import poisson


<font size="3">1.Import the tables</font> 

In [54]:
tables=pickle.load(open('../Web_Scraping /Data/dict_table','rb'))
history=pd.read_csv('../Web_Scraping /Data/Clean_Fifa_worldcup_missing_data.csv')
fixture=pd.read_csv('../Web_Scraping /Data/Clean_Fixture_2022.csv')
history.rename({'HomeAway':'AwayTeam'},axis=1,inplace=True)

<font size="6">2.Calculate Team Strength..</font>

In [29]:
df_home=history[['HomeTeam','HomeGoals','AwayGoals']]
df_away=history[['AwayTeam','AwayGoals','HomeGoals']]
df_home=df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away=df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

'

<font size="3">Concatenate df_home and df_away make a groupby by 'Team' and calculate the average..</font> 

In [49]:
df_team_strength=pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()

<font size="3">This model was chosen because it fits the Poisson distribution</font> 

<img src="../source/Condiciones.png">

<font size="3">So.. how works?</font> 

<img src="../source/Distribucion de poisson.png">

<font size="3">Having understood the formula. let's apply it to our example!!</font> 

In [35]:
def predict_point(home,away):
    if home in df_team_strength.index and away in df_team_strength.index:
        #goals scored* goals conceded
        lamb_home=df_team_strength.at[home,'GoalsScored']*df_team_strength.at[away,'GoalsConceded']
        lamb_away=df_team_strength.at[away,'GoalsScored']*df_team_strength.at[home,'GoalsConceded']
        prob_home,prob_away,prob_draw=0,0,0
        for x in range(0,11):#Numbers of Goals of Home Team
            for y in range(0,11):#Numbers of Goals of Away Team
                p=poisson.pmf(x,lamb_home)*poisson.pmf(y,lamb_away)
                if x==y:
                    prob_draw+=p
                elif x>y:
                    prob_home+=p
                else:
                    prob_away+=p
        points_home=3*prob_home+prob_draw
        points_away=3*prob_away+prob_draw
        return (points_home,points_away)
    else:
        return(0,0)

<font size="3">Let's test the function! </font> 

In [48]:
predict_point('Bolivia','Argentina')

(0.010097814248235259, 2.894594744185821)

<font size="6">3.Predicting the world cup 2022</font>

In [57]:
fixture_48=fixture[:48].copy()
fixture_knockout=fixture[48:56].copy()
fixture_quarter=fixture[56:60].copy()
fixture_semi=fixture[60:62].copy()
fixture_final=fixture[62:].copy()


,home,score,away,year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
5,Netherlands,Match 36,Qatar,2022
6,England,Match 3,Iran,2022
7,United States,Match 4,Wales,2022
8,Wales,Match 17,Iran,2022
9,England,Match 20,United States,2022
